In [ ]:
import matplotlib.pyplot as plt
import cyipopt as ipopt
import numpy as np

from vessel_models import VesselModel
from controller import MPC

In [ ]:
# Simulation variables
N = 120
dt = 0.5
ns = 6
nu = 3

N_mpc = 30 # default 10
N_sim = N + N_mpc

# Cost matrices
Qvec = 3*np.ones(ns) # default 3
Rvec = 0.1*np.ones(nu)
Qfvec = 10*np.ones(ns)

# Initial and goal states
sinit = np.array([0, 0, 0, 0, 0, 0])
sgoal = np.array([10, 10, np.pi/2, 0, 0, 0])

# Obstacle States
num_obstacles = 1
obstacles = np.zeros((num_obstacles,3))
obstacles[0] = [5, 5, 2]

model = VesselModel(mismatch=False)
print(model.m)

In [ ]:
U_soln = np.zeros((N-1, nu))
S_soln = np.zeros((N, ns))
S_soln[0] = sinit

for i in range(N-1):
    s0 = S_soln[i]
    sf = sgoal
    sref = sgoal
    trajoptimizer = MPC(model, N_mpc, ns, nu, dt, Qvec, Rvec, Qfvec, s0, sf, sref, obstacles)
    trajoptimizer.exec_MPC(version="without_traj")
    S, U = trajoptimizer.flat2vec(trajoptimizer.sol.x)
    U_soln[i] = U[0]
    S_soln[i+1] = model.rk4(s0, U[0], dt)

In [ ]:
Sfwd = np.zeros((N,ns))
Sfwd[0] = sinit
for i in range(N-1):
    Sn = Sfwd[i]
    Un = U_soln[i]
    Sfwd[i+1] = model.rk4(Sn, Un, dt)

ps = Sfwd.T
ps_soln = S_soln.T
fig, axes = plt.subplots(1, 1, figsize=(20, 20))

for obstacle in obstacles:
    obs_x, obs_y, obs_r = obstacle
    circle = plt.Circle((obs_y, obs_x), obs_r, color='r')
    axes.add_patch(circle)

for i in range(len(ps[1])):
    dx = 0.01*np.cos(ps[2,i])
    dy = 0.01*np.sin(ps[2,i])
    axes.arrow(ps[1,i], ps[0, i], dy, dx, head_width=0.005, head_length=0.005, fc='green', ec='green')

axes.scatter(ps_soln[1], ps_soln[0])
axes.scatter(ps[1], ps[0])
axes.set_aspect('equal', adjustable='box')
plt.show()